In [28]:
import numpy as np
from transformers import BertConfig,BertModel,AutoConfig,AutoTokenizer,BertForNextSentencePrediction
import torch
from torch.nn.functional import softmax
from datasets import load_dataset,load_from_disk
import json


In [5]:

#dataset = load_dataset('trivia_qa', 'rc')
#dataset.save_to_disk('.')

In [8]:
dataset = load_from_disk(r'\\wsl$\Ubuntu-20.04\home\jolteon\NLUProject\data\trivia_qa_rc')

In [ ]:
train_dataset = dataset['train']

In [38]:
def function(item):
    print(len(item['search_results']['search_context']))

In [41]:
#train_dataset.map(function)

# Useful fields: 
Question:
Answer:
Entity_pages -> wiki_context (multiple contexts)
search_results -> search_context (multiple contexts)

# Planning:
set of documents which contains all of wiki context and search results

for each document:
    Split by sentences using NLTK:
    Tokenize each sentence using bert (wordpiece) embeddings
    Store each tokenized sentence in a ordered dict of {sentence_idx -> tokenized sentence}
    run Bert's NSP over the document to generate sequence of split confidence
    Pick a confidence threshold. Threshold needs to be s.t each segment has less than 512 tokens.
    Assign each sentence to a segment based on threshold.
    
    
    


In [48]:
for ii,item in enumerate(train_dataset):
    #print(json.dumps(item,indent=4))
    print(len(item['search_results']['search_context']), len(item['entity_pages']['wiki_context']))
    if ii == 7:
         print(json.dumps(item,indent=4))
    if ii ==10:
        break
    

7 0
6 2
2 0
7 3
4 1
6 1
1 2
0 1
{
    "answer": {
        "aliases": [
            "\u65e5\u672c\u570b",
            "State of Japan",
            "Ja-pan",
            "Nihon",
            "Nippon",
            "Japang",
            "Modern\u2013era Japan",
            "Etymology of Japan",
            "Japan (country)",
            "Republic of Japan",
            "Japanese financial sector",
            "Nihon-koku",
            "ISO 3166-1:JP",
            "\u65e5\u672c",
            "Japian",
            "Japan",
            "JapaN",
            "The State of Japan",
            "Riben",
            "Nippon-koku",
            "JPN",
            "Jpan",
            "R\u00ecb\u011bn",
            "\u65e5\u672c\u56fd",
            "Japao",
            "JAPAN",
            "Japoa",
            "\ud83d\uddfe"
        ],
        "matched_wiki_entity_name": "",
        "normalized_aliases": [
            "nihon koku",
            "nihon",
            "japoa",
            "japang",
      

In [ ]:
for item in train_dataset:
    print(json.dumps(item,indent=4))
    #print(item['search_result']['search_context'])
    break

In [11]:
print(torch.cuda.is_available())

True


In [12]:
model_name = 'bert-base-cased'

tokenizer = AutoTokenizer.from_pretrained(model_name)
#config = AutoConfig.from_pretrained(model_name)
model = BertForNextSentencePrediction.from_pretrained(model_name)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
seq_A = 'I like cookies !'
seq_B = 'Do you like them ?'
encoded = tokenizer.encode_plus(seq_A, text_pair=seq_B, return_tensors='pt')


In [14]:
outputs = model(**encoded, labels=torch.LongTensor([1]))
probs = softmax(outputs.logits, dim=1)
prob_next = probs[0][0]